In [1]:
import nltk
import re
import codecs
import csv
import os
from sklearn import metrics

path = 'D:\Dropbox\ongoing\chrisIntervention'

In [2]:
# read cohort info from the CSV and save them with the sbj ID

intvIdx = 'Ex4'

category = {}
with open(os.path.join(path, intvIdx + '_key.csv')) as f:
    sbjData = csv.reader(f)
    next(sbjData) # skip the header
    for row in sbjData:
        category[row[0]] = int(row[1])

subjectId = category.keys()        
        
#print(subjectId)
#print()
print('# of cohort 1: ', sum(1 for sbj in category if category[sbj] == 1))
print('# of cohort 2: ', sum(1 for sbj in category if category[sbj] == 2))


# of cohort 1:  153
# of cohort 2:  202


In [3]:
# build the dictionary based on the training set
# this is just to check if it is working
# we should build the dictionary for each leave-one-out test

def make_dictionary(subjectPool, critNumSbj):
    allWords = []
    for sbj in subjectPool:
        sbjWords = []
        for line in codecs.open(os.path.join(path, intvIdx, sbj + '.txt'), 'r' , 'utf-8'):
            line = line.strip()
            line = line.lower()
            tokens = nltk.wordpunct_tokenize(line)
            text = nltk.Text(tokens)
            words = [w for w in text if re.search('[a-z]+',w) and len(w)>2]
            sbjWords.extend(words)
        # when making dictionary, make the words from a document unique
        sbjWords = list(set(sbjWords))
        allWords.extend(sbjWords)
    
    # once we make list of all words, we count the words
    cntWords = nltk.FreqDist(allWords)
    #for ii in range(21,0,-2):
    #    print('# words with freq >', ii, ' : ', sum(1 for word in cntWords if cntWords[word] > ii))
    
    # we go with the words that came from at least N different people (arbitrary) 
    # we can change and see what happens ...
    return [word for word in cntWords if cntWords[word] > critNumSbj]

dictionary = make_dictionary(subjectId, 5)

print(dictionary)

['probably', 'spending', 'too', 'example', 'cousins', 'watch', 'take', 'parents', 'camping', 'sport', 'listening', 'almost', 'the', 'creative', 'when', 'since', 'playing', 'practice', 'weekend', 'walking', 'clothes', 'before', 'put', 'how', 'times', 'makes', 'different', 'having', 'with', 'use', 'into', 'swim', 'but', 'around', 'pool', 'this', 'ways', 'getting', 'most', 'right', 'even', 'mom', 'learn', 'chicago', 'friends', 'just', 'get', 'was', 'cool', 'keep', 'don', 'did', 'play', 'change', 'first', 'laugh', 'feel', 'better', 'see', 'walk', 'your', 'biking', 'talk', 'lot', 'foot', 'done', 'swimming', 'need', 'through', 'those', 'out', 'their', 'things', 'though', 'rap', 'tennis', 'any', 'helping', 'teams', 'winter', 'cold', 'bike', 'hope', 'start', 'during', 'good', 'food', 'sing', 'birthday', 'stay', 'trips', 'nba', 'game', 'one', 'brother', 'songs', 'enjoy', 'and', 'park', 'people', 'that', 'kind', 'because', 'other', 'played', 'these', 'home', 'days', 'come', 'look', 'after', 'any

In [4]:
# we define a function that tunrs text into a list of tokens

def extract_feature(text, dictionary):
    text = text.strip()
    text = text.lower()
    tokens = nltk.wordpunct_tokenize(text)
    text = nltk.Text(tokens)
    feature = {}
    for word in dictionary:
        feature[word] = (word in text)
    return feature

print('nltk output: ', nltk.wordpunct_tokenize('You think you can do this one, Kate?'))
print(extract_feature('You think you can do this one, Kate?', dictionary))

nltk output:  ['You', 'think', 'you', 'can', 'do', 'this', 'one', ',', 'Kate', '?']
{'tennis': False, 'really': False, 'always': False, 'any': False, 'everyday': False, 'everything': False, 'ride': False, 'helping': False, 'gives': False, 'another': False, 'probably': False, 'best': False, 'spending': False, 'him': False, 'sister': False, 'chicago': False, 'too': False, 'example': False, 'pop': False, 'those': False, 'hip': False, 'teams': False, 'way': False, 'lot': False, 'friends': False, 'because': False, 'cousins': False, 'good': False, 'cold': False, 'lots': False, 'inside': False, 'bike': False, 'now': False, 'dont': False, 'warmer': False, 'focus': False, 'right': False, 'hope': False, 'just': False, 'watch': False, 'thats': False, 'isn': False, 'myself': False, 'might': False, 'eat': False, 'take': False, 'start': False, 'parents': False, 'they': False, 'matter': False, 'its': False, 'and': False, 'many': False, 'camping': False, 'sport': False, 'fishing': False, 'while': Fals

In [5]:
# leave-one-out classification

predLabel = []
trueLabel = []
currSbj = 1;

for sbj in subjectId:
    # sbj is being tested, others are used to train the classifier
    trainSbj = list(subjectId)
    trainSbj.remove(sbj)
    # create the dictionary
    trainDict = make_dictionary(trainSbj, 5)
    trainText = []
    # create the training set
    for tSbj in trainSbj:
        tmpText = ''
        for line in codecs.open(os.path.join(path, intvIdx, tSbj + '.txt'), 'r' , 'utf-8'):
            tmpText = tmpText + ' ' + line
        trainText.append([extract_feature(tmpText, trainDict), category[tSbj]])
    # train classifier
    classifier = nltk.NaiveBayesClassifier.train(trainText)

    # now the test set
    trueLabel.append( category[sbj] )
    testText = ''
    for line in codecs.open(os.path.join(path, intvIdx, sbj + '.txt'), 'r' , 'utf-8'):
        testText = testText + ' ' + line
    # predict!
    currPred = classifier.classify(extract_feature(testText, trainDict))
    predLabel.append( currPred )
    
    print(str(currSbj) + ' (' + sbj + '): pred - ' + str(currPred) + ', actual - ' + str(category[sbj]) )
    currSbj = currSbj + 1

print('done. ')


1 (310076_4): pred - 1, actual - 1
2 (920078_4): pred - 2, actual - 2
3 (410316_4): pred - 1, actual - 1
4 (110017_4): pred - 2, actual - 1
5 (520056_4): pred - 2, actual - 2
6 (1120051_4): pred - 1, actual - 2
7 (120045_4): pred - 2, actual - 2
8 (920109_4): pred - 1, actual - 2
9 (320110_4): pred - 2, actual - 2
10 (320035_4): pred - 2, actual - 2
11 (520164_4): pred - 2, actual - 2
12 (120064_4): pred - 2, actual - 2
13 (210716_4): pred - 2, actual - 1
14 (810422_4): pred - 1, actual - 1
15 (220117_4): pred - 2, actual - 2
16 (320078_4): pred - 2, actual - 2
17 (1020110_4): pred - 2, actual - 2
18 (1020070_4): pred - 2, actual - 2
19 (220110_4): pred - 2, actual - 2
20 (610521_4): pred - 2, actual - 1
21 (610408_4): pred - 2, actual - 1
22 (720026_4): pred - 2, actual - 2
23 (510717_4): pred - 2, actual - 1
24 (620103_4): pred - 2, actual - 2
25 (1120071_4): pred - 2, actual - 2
26 (720135_4): pred - 2, actual - 2
27 (910316_4): pred - 1, actual - 1
28 (211101_4): pred - 2, actual -

In [6]:
# summarize the results

print(metrics.classification_report(trueLabel, predLabel))


             precision    recall  f1-score   support

          1       0.79      0.59      0.67       153
          2       0.74      0.88      0.80       202

avg / total       0.76      0.75      0.75       355



In [7]:
# ok, it is POSSIBLE TO PREDICT cohort from the writings.
# now, show me the informative features

dictionary = make_dictionary(subjectId, 5)
print(len(dictionary))

allText = []

for sbj in subjectId:
    tmpText = ''
    for line in codecs.open(os.path.join(path, intvIdx, sbj + '.txt'), 'r' , 'utf-8'):
        tmpText = tmpText + ' ' + line
        allText.append([extract_feature(tmpText, dictionary), category[sbj]])

# train classifier based on all information
classifier = nltk.NaiveBayesClassifier.train(allText)


324


In [8]:
classifier.show_most_informative_features(100)


Most Informative Features
                  matter = True                1 : 2      =      9.2 : 1.0
                   after = True                1 : 2      =      8.3 : 1.0
                  better = True                1 : 2      =      7.6 : 1.0
                  inside = True                1 : 2      =      7.5 : 1.0
                   learn = True                1 : 2      =      6.6 : 1.0
                     nba = True                1 : 2      =      6.6 : 1.0
                  starts = True                1 : 2      =      6.6 : 1.0
                   never = True                1 : 2      =      6.6 : 1.0
                  warmer = True                2 : 1      =      5.8 : 1.0
                    ones = True                1 : 2      =      5.7 : 1.0
                     win = True                1 : 2      =      5.7 : 1.0
                 flowers = True                1 : 2      =      5.7 : 1.0
                  change = True                1 : 2      =      5.7 : 1.0